# Preparation

Here is where we will keep up with all the packages we need throughout the project.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard # for visualizing our models
from sklearn.utils import class_weight
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from PIL import Image
import shutil
import cv2
import random
import pickle
import time

# Let's first prepare the data set for use
In it's current state, the pokemon images are not sorted by the classes which we want to predict for. All 809 images are in the same folder, and some are of different extensions ie. png vs jpg. This is rather unuseful for creating our model. We will sort them using the csv file which contains their types, by reading this into a pandas dataframe, then creating the folders for each type and moving the images into their corresponding folders all as jpgs. We will only be sorting and creating our model with Type1 in mind, since there are way too many instances when Type2 is null. Please note that this code expect your notebook to be in the same folder as the data folder.

In [3]:
poke_img_dir = 'data/images/images/'
poke_csv = 'data/pokemon.csv'
poke_df = pd.read_csv(poke_csv)
del poke_df['Type2'] # remove Type2 since we will not be working with it
poke_classes = poke_df.Type1.unique().tolist()
print(poke_classes)

['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric', 'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice', 'Dragon', 'Dark', 'Steel', 'Flying']


Now that we have our classes, we need to create a folder for each and iterate through each pokemon image, copying them into their corresponding folder and converting them to .jpgs simultaneously for uniformity.

In [58]:
for poke_class in poke_classes:
    class_dir = poke_img_dir + poke_class
    os.mkdir(class_dir)
    pokes = poke_df.loc[poke_df['Type1'] == poke_class] # get all pokemon for the current class
    pokes = pokes.Name # we just need the names from the series
    for poke in pokes:
        search_str = poke_img_dir + poke + '.*' # string input for glob to find poke img file
        poke_img = glob.glob(search_str)
        poke_img = str(poke_img[0]) # convert file in list from glob search to string
        class_folder = poke_img_dir + poke_class + '/' + poke + '.jpg' # convert all to jpg for uniformity
        shutil.copy(poke_img, class_folder) # copy the image into class folder
    print(poke_class + ' pokemons copied into corresponding folder.')

Grass pokemons copied into corresponding folder.
Fire pokemons copied into corresponding folder.
Water pokemons copied into corresponding folder.
Bug pokemons copied into corresponding folder.
Normal pokemons copied into corresponding folder.
Poison pokemons copied into corresponding folder.
Electric pokemons copied into corresponding folder.
Ground pokemons copied into corresponding folder.
Fairy pokemons copied into corresponding folder.
Fighting pokemons copied into corresponding folder.
Psychic pokemons copied into corresponding folder.
Rock pokemons copied into corresponding folder.
Ghost pokemons copied into corresponding folder.
Ice pokemons copied into corresponding folder.
Dragon pokemons copied into corresponding folder.
Dark pokemons copied into corresponding folder.
Steel pokemons copied into corresponding folder.
Flying pokemons copied into corresponding folder.


Now our data is sorted and ready to work with.

# Create training and validation sets

In [121]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2, rescale = 1.0/255 
    # split specifies 80% will got to training set and 20% to validation set
)
training_set = train_datagen.flow_from_directory(
    poke_img_dir,
    target_size = (120, 120), 
    color_mode = 'rgba', # rgba accounts for transparency
    classes = poke_classes,
    class_mode = 'categorical', 
    batch_size = 20, 
    shuffle = True,
    seed = 7, # random seed
    subset = 'training'
)
validation_set = train_datagen.flow_from_directory(
    poke_img_dir,
    target_size = (120, 120), 
    color_mode = 'rgba',
    classes = poke_classes,
    class_mode = 'categorical', 
    batch_size = 20, 
    shuffle = True,
    seed = 7, #
    subset = 'validation'
)

Found 656 images belonging to 18 classes.
Found 153 images belonging to 18 classes.


Next let's find the weights of each class to account for imbalance.

In [126]:
print(training_set.class_indices, '\n')
weights = class_weight.compute_class_weight(
    'balanced',
     np.unique(training_set.classes),
     training_set.classes
)
classes = np.unique(training_set.classes)
class_weights = dict(zip(classes, weights))
print(class_weights)

{'Grass': 0, 'Fire': 1, 'Water': 2, 'Bug': 3, 'Normal': 4, 'Poison': 5, 'Electric': 6, 'Ground': 7, 'Fairy': 8, 'Fighting': 9, 'Psychic': 10, 'Rock': 11, 'Ghost': 12, 'Ice': 13, 'Dragon': 14, 'Dark': 15, 'Steel': 16, 'Flying': 17} 

{0: 0.5784832451499118, 1: 0.8475452196382429, 2: 0.3961352657004831, 3: 0.6283524904214559, 4: 0.43386243386243384, 5: 1.3015873015873016, 6: 1.1388888888888888, 7: 1.4017094017094016, 8: 2.4296296296296296, 9: 1.5185185185185186, 10: 0.8475452196382429, 11: 0.984984984984985, 12: 1.6565656565656566, 13: 1.9181286549707601, 14: 1.6565656565656566, 15: 1.5185185185185186, 16: 1.7354497354497354, 17: 12.148148148148149}


# Creating the model

In [134]:
# inputs are going to be 120x120 images with 4 color channels (transparency)
input_shape = (120, 120, 4)
model_inputs = tf.keras.Input(shape = input_shape)

# first 2d convolution layer
c1 = tf.keras.layers.Conv2D(
    filters = 30, # the number of features, low level
    kernel_size = 4, # size of the sliding window 4x4
    activation = 'relu' # activation function
)(model_inputs)
# first pooling layer
p1 = tf.keras.layers.MaxPool2D(
    pool_size = 2, # window size over which to take the maximum
    strides = 2 # specifies how far pooling window moves for each pooling step
)(c1)

# second 2d convolution layer
c2 = tf.keras.layers.Conv2D(
    filters = 60, # mid level
    kernel_size = 4,
    activation = 'relu'
)(p1)
# second pooling layer
p2 = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 2
)(c2)

# third 2d convolution layer
c3 = tf.keras.layers.Conv2D(
    filters = 120, # high level
    kernel_size = 4,
    activation = 'relu'
)(p2)
# third pooling layer
p3 = tf.keras.layers.MaxPool2D(
    pool_size = 2,
    strides = 2
)(c3)

model_outputs = tf.keras.layers.GlobalAveragePooling2D()(p3) #changed to p2

poke_model = tf.keras.Model(
    name = 'PokemonModel',
    inputs = model_inputs,
    outputs = model_outputs
)
poke_model.summary()

Model: "PokemonModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 120, 120, 4)]     0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 117, 117, 30)      1950      
_________________________________________________________________
max_pooling2d_90 (MaxPooling (None, 58, 58, 30)        0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 55, 55, 60)        28860     
_________________________________________________________________
max_pooling2d_91 (MaxPooling (None, 27, 27, 60)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 24, 24, 120)       115320    
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 12, 12, 120)      

# Classifying

In [129]:
classifier_inputs = poke_model.input # same inputs as the model
classifier_outputs = tf.keras.layers.Dense(
    units = 18,  # 18 neurons for categorical classification
    activation = 'sigmoid' # sigmoid function sigmoid(x) = 1 / (1 + exp(-x))
)(poke_model.output)

poke_classifier = tf.keras.Model(
    name = 'PokemonClassifier',
    inputs = classifier_inputs,
    outputs = classifier_outputs
)

poke_classifier.summary()

Model: "PokemonClassifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 120, 120, 4)]     0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 117, 117, 120)     7800      
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 58, 58, 120)       0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 55, 55, 120)       230520    
_________________________________________________________________
max_pooling2d_85 (MaxPooling (None, 27, 27, 120)       0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 24, 24, 120)       230520    
_________________________________________________________________
max_pooling2d_86 (MaxPooling (None, 12, 12, 120) 

# Training the model

Now we will begin testing our model. We'll use tensorboard to visualize how well it is working.

In [131]:
log_name = 'Poke-Classifier-CNN-{}-all120'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(log_name))

poke_classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = poke_classifier.fit(
    training_set,
    validation_data = validation_set,
    epochs = 30,
    class_weight = class_weights,
    callbacks = [tensorboard]
)

Epoch 1/30
33/33 [==============================] - 51s 2s/step - loss: 1.8386 - accuracy: 0.3443 - val_loss: 2.9451 - val_accuracy: 0.1176
Epoch 2/30
33/33 [==============================] - 53s 2s/step - loss: 1.7046 - accuracy: 0.3437 - val_loss: 2.8540 - val_accuracy: 0.1830
Epoch 3/30
33/33 [==============================] - 40s 1s/step - loss: 1.6844 - accuracy: 0.3598 - val_loss: 2.9532 - val_accuracy: 0.1569
Epoch 4/30
33/33 [==============================] - 40s 1s/step - loss: 1.6001 - accuracy: 0.4008 - val_loss: 2.7972 - val_accuracy: 0.1634
Epoch 5/30
33/33 [==============================] - 43s 1s/step - loss: 1.5943 - accuracy: 0.4329 - val_loss: 2.9324 - val_accuracy: 0.1503
Epoch 6/30
33/33 [==============================] - 48s 1s/step - loss: 1.5171 - accuracy: 0.4387 - val_loss: 2.8816 - val_accuracy: 0.1503
Epoch 7/30
33/33 [==============================] - 47s 1s/step - loss: 1.4476 - accuracy: 0.4024 - val_loss: 2.9876 - val_accuracy: 0.1438
Epoch 8/30
33/33 [==

KeyboardInterrupt: 

# Extras

Below is a scrapped version of how I first built my model

In [7]:
image_sample = training_set.next()
image_sample[0].shape
#training_set = []
#for poke_class in poke_classes:
    #class_dir = os.path.join(poke_img_dir, poke_class) # path to type folders
    #class_num = poke_classes.index(poke_class) # arbitrary classification value from index in array
    #for poke_img in os.listdir(class_dir):
        #poke_img_array = cv2.imread(os.path.join(class_dir, poke_img), cv2.IMREAD_UNCHANGED) # unchanged includes the alpha channel which is transparency 
        #rs_poke_img_array = cv2.resize(poke_img_array, (img_size, img_size)) # resized to 75x75
        #training_set.append([rs_poke_img_array, class_num])
#random.shuffle(training_set)
#for sample in training_set[:20]:
    #print(sample[1])
# shows the class number for the first 20 entries, the set appears shuffled
# Now that the set is shuffled, we can generate the variables we will use as input into our CNN.
X = [] # features set
y = [] # labels
for features, label in training_set:
    X.append(features)
    y.append(label)
# for use with keras X and y must be converted to numpy arrays
X = np.array(X)
X= X.reshape(-1, img_size, img_size, 3) # -1 to specify any amount of features, 3 RGB values for color
y = np.array(y)
#In order to save time later when making slight adjustments to our model, we'll use pickle to save our data.
pickle_o = open("X.pickle", "wb")
pickle.dump(X, pickle_o)
pickle_o.close()

pickle_o = open("y.pickle", "wb")
pickle.dump(y, pickle_o)
pickle_o.close()
# test to ensure pickle worked

#Now that we've completed our pre-processing, we can start to build or convolutional nerual network. For more information on how a CNN functions, please see my report for this project.

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

max_pixel = 255.0
X = X/max_pixel # normalize the pixel values to a range between 0 and 1 to speed up training

#create sequential model and add layers
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:])) # 64 units, (3,3) windows size
model.add(Activation("relu")) # rectify linear
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3))) # 1: ignores the -1 we inputted into reshape
model.add(Activation("relu")) # rectify linear
model.add(MaxPooling2D(pool_size=(2,2)))

# flatten the data set to 1D from 2D for to add dense layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

#before we begin training our model, we need to ensure it has the correct weighting for our classes
labels, counts = np.unique(y, return_counts = True)
lcs = np.asarray((labels, counts)).T # combine labels and counts into one array for iteration
total = y.size # total of all counts from labels is the same as y.size
weights = [] # the weights of the classes in the order in which they are in the labels array
for lc in lcs:
    label = lc[0]
    count = lc[1]
    weight = (1 / count) * (total)/2.0 # Scaling by total/2 helps keep the loss to a similar magnitude.
    weights.append(weight)
class_weight = dict(zip(labels, weights)) # combine the labels and weights arrays into one dict
print(class_weight)
print(poke_classes)

model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])
model.fit(X, y, epochs = 10, batch_size = 16, class_weight=class_weight, validation_split = 0.1) # best to have batch size somewhere in 20s

(32, 120, 120, 4)